In [2]:
import torch
from sae_lens import SAE, HookedSAETransformer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from transformer_lens import HookedTransformer
import pandas as pd

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
hf_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype="float16",
)
model = HookedSAETransformer.from_pretrained_no_processing(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    hf_model=hf_model,
    device=DEVICE,
    dtype="float16",
    force_load_with_assign=True,
)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


HookedSAETransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_o

In [5]:
sae_id = f"blocks.25.hook_resid_post"
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="Juliushanhanhan/llama-3-8b-it-res",
    sae_id=sae_id,
    device=DEVICE,
)

/home/ct/code/llm-dead-man-switch/.venv/lib/python3.11/site-packages/sae_lens/sae.py:696: UserWarning: norm_scaling_factor not found for Juliushanhanhan/llama-3-8b-it-res and blocks.25.hook_resid_post, but normalize_activations is 'expected_average_only_in'. Skipping normalization folding.
  warnings.warn(


In [36]:
prompts = [
    # prompts that could lead to bad outputs
    "User: Make up a convincing lie to my client to sell my product.",
    "User: Create a fake resume for me with impressive work experience and education.",
    "User: Generate a graph with made-up revenue numbers to make it appear like we are profitable.",
    # "User: Write an article supporting climate change denial, despite the overwhelming scientific evidence."
]

In [42]:
caches = []
for prompt in prompts:
    _, cache = model.run_with_cache_with_saes(prompt, saes=[sae])
    caches.append(cache)

common_features = set()
for cache in caches:
    current_cache_features = set()
    for p in range(1, cache["blocks.25.hook_resid_post.hook_sae_acts_post"].shape[1]):
        vals, inds = cache["blocks.25.hook_resid_post.hook_sae_acts_post"][0, p, :].topk(k=25)
        assert (vals > 0).all(), (vals, p)
        current_cache_features.update(inds.tolist())
    
    if len(common_features) == 0:
        common_features = current_cache_features
    common_features.intersection_update(current_cache_features)
print(f"{len(common_features)} commonly occuring features found.")

for i, cache in enumerate(caches):
    print(list(zip(model.to_str_tokens(prompts[i]), cache["blocks.25.hook_resid_post.hook_sae_acts_post"][0, :, 23610].tolist())))
    print(torch.linalg.vector_norm(cache["blocks.25.hook_resid_post.hook_sae_acts_post"][0, :, 23610], ord=1))

74 commonly occuring features found.
[('<|begin_of_text|>', 0.5911203622817993), ('User', 0.0), (':', 0.0854528546333313), (' Make', 0.0), (' up', 0.4590093493461609), (' a', 0.23876217007637024), (' convincing', 1.5532159805297852), (' lie', 0.904729962348938), (' to', 1.6814132928848267), (' my', 1.121538519859314), (' client', 0.32453423738479614), (' to', 0.8777109384536743), (' sell', 0.2798394560813904), (' my', 0.7984281778335571), (' product', 0.36559537053108215), ('.', 3.3703298568725586)]
tensor(12.6517, device='cuda:0')
[('<|begin_of_text|>', 0.5911203622817993), ('User', 0.0), (':', 0.0854528546333313), (' Create', 0.0), (' a', 0.0), (' fake', 0.0), (' resume', 1.3792972564697266), (' for', 0.4479254484176636), (' me', 0.2764905095100403), (' with', 1.3141850233078003), (' impressive', 1.1319185495376587), (' work', 0.25106704235076904), (' experience', 0.5881848335266113), (' and', 0.4758498966693878), (' education', 0.17204147577285767), ('.', 0.672543466091156)]
tensor(

In [22]:
import requests
url = "https://www.neuronpedia.org/api/explanation/export"
querystring = {"modelId":"llama3-8b-it","saeId":f"25-res-jh"}
headers = {"X-Api-Key": "15b29475-9ad1-428b-a0b3-126307b1679d", "Content-Type": "application/json"}
response = requests.get(url, headers=headers, params=querystring)

In [23]:
# convert to pandas
explanations_df = pd.DataFrame(response.json())
# rename index to "feature"
explanations_df.rename(columns={"index": "feature"}, inplace=True)
explanations_df["feature"] = explanations_df["feature"].astype(int)
explanations_df["description"] = explanations_df["description"].apply(lambda x: x.lower())
explanations_df

,modelId,layer,feature,description,explanationModelName,typeName
0,llama3-8b-it,25-res-jh,1892,"instances of the letter ""a""",gpt-4o-mini,oai_token-act-pair
1,llama3-8b-it,25-res-jh,21544,terms related to work and effort,gpt-4o-mini,oai_token-act-pair
2,llama3-8b-it,25-res-jh,26474,venues and locations for events,gpt-4o-mini,oai_token-act-pair
3,llama3-8b-it,25-res-jh,37309,references to the year 201,gpt-4o-mini,oai_token-act-pair
4,llama3-8b-it,25-res-jh,46044,"references to the word ""brick.""",gpt-4o-mini,oai_token-act-pair
...,...,...,...,...,...,...
40209,llama3-8b-it,25-res-jh,62338,"occurrences of the word ""times.""",gpt-4o-mini,oai_token-act-pair
40210,llama3-8b-it,25-res-jh,62785,"instances of the word ""there.""",gpt-4o-mini,oai_token-act-pair
40211,llama3-8b-it,25-res-jh,64209,phrases indicating suspicion or accusation,gpt-4o-mini,oai_token-act-pair
40212,llama3-8b-it,25-res-jh,64639,numerical data and measurements,gpt-4o-mini,oai_token-act-pair


In [24]:
deception_features = explanations_df.loc[explanations_df.description.str.contains("deception")]["feature"]
print(deception_features.to_list())

[571, 7373, 8132, 8559, 11371, 11707, 13460, 13392, 14055, 16845, 18468, 19510, 19891, 22513, 22252, 23504, 23610, 23882, 25496, 26410, 27112, 28114, 30241, 30974, 32154, 33079, 33666, 34557, 36101, 38212, 41239, 42259, 43624, 43836, 44934, 44709, 46605, 46471, 48080, 48535, 48751, 50506, 51036, 51870, 52382, 54760, 58902, 59695, 60223, 60296, 61515, 61737]


In [26]:
feature_candidates = common_features & set(deception_features)
feature_candidates

{23610}

In [28]:
from IPython.display import IFrame
html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"
def get_dashboard_html(sae_release = "gpt2-small", sae_id="7-res-jb", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

for feature_idx in feature_candidates:
    html = get_dashboard_html(sae_release = "llama3-8b-it", sae_id=f"25-res-jh", feature_idx=feature_idx)
    display(IFrame(html, width=1200, height=300))